### 流实现的行为方式  

In [ ]:
from base_definition import cons_stream, car_stream, cdr_stream


def enumerate_interval_stream(start, end):
    if start > end:
        return None
    return cons_stream(start, lambda: enumerate_interval_stream(start + 1, end))


stream = enumerate_interval_stream(8, 100)


def filter_stream(pred, stream):
    if stream is None:
        return None
    if pred(car_stream(stream)):
        return cons_stream(
            car_stream(stream), lambda: filter_stream(pred, cdr_stream(stream))
        )
    return filter_stream(pred, cdr_stream(stream))


def is_primer(n):
    def square(x):
        return x * x

    def smallest_divisor(n):
        def find_divisor(n, test_divisor):
            if square(test_divisor) > n:
                return n
            if n % test_divisor == 0:
                return test_divisor
            return find_divisor(n, test_divisor + 1)

        return find_divisor(n, 2)

    return smallest_divisor(n) == n


f_stream = filter_stream(is_primer, stream)
print(car_stream(f_stream))
print(car_stream(cdr_stream(f_stream)))

### memo_proc

In [ ]:
from base_definition import cons_stream, car_stream, cdr_stream


def memo_proc(proc):
    already_run = False
    result = False

    def sub_func():
        nonlocal already_run
        nonlocal result
        if already_run:
            return result
        else:
            result = proc()
            already_run = True
            return result

    return sub_func


f = lambda: 1
print(memo_proc(f)())
print(memo_proc(f)())

### 3.50 
1. one parameter map stream
2. multi parameters map stream

In [ ]:
from base_definition import cons_stream, cdr_stream, car_stream, enumerate_interval_stream


def map_stream(proc, stream):
    if stream is None:
        return None
    return cons_stream(proc(car_stream(stream)), lambda: map_stream(proc, cdr_stream(stream)))


stream = enumerate_interval_stream(2, 7)
m_stream = map_stream(lambda x: x * x, stream)
print(car_stream(m_stream))
print(car_stream(cdr_stream(m_stream)))

In [ ]:
from base_definition import (
    cons_stream,
    cdr_stream,
    car_stream,
    enumerate_interval_stream,
)

def map_stream(proc, *stream):
    params = []
    next_stream = []
    for arg in stream:
        if arg is None:
            return
        params.append(car_stream(arg))
        next_stream.append(cdr_stream(arg))
    return cons_stream(proc(*params), lambda : map_stream(proc, *next_stream))


s1 = enumerate_interval_stream(1, 1)
s2 = enumerate_interval_stream(1, 1)
s = map_stream(lambda x, y: x + y, s1, s2)
print(car_stream(s))
print(cdr_stream(s))

### 3.51 

In [ ]:
from base_definition import ref_stream, map_stream, enumerate_interval_stream
stream = map_stream(print, enumerate_interval_stream(1, 10))
ref_stream(stream, 5)
ref_stream(stream, 7)




### 3.52 

In [ ]:
from base_definition import (
    enumerate_interval_stream,
    map_stream,
    filter_stream,
    cdr_stream,
    car_stream,
    ref_stream,
    display_stream
)

sum = 0


def accum(x):
    global sum
    sum += x
    return sum


def is_even(x):
    return x % 2 == 0


stream = map_stream(accum, enumerate_interval_stream(1, 5))
display_stream(stream)
print(sum)
display_stream(stream)
print(sum)
y = filter_stream(is_even, stream)
display_stream(y)
z = filter_stream(lambda x: x % 5 == 0, stream)
print(car_stream(y))
print(car_stream(z))
print(ref_stream(y, 7))
print(sum)

### 所有不能被7整除的整数无穷流 

In [ ]:
from base_definition import (
    cons_stream,
    delay,
    filter_stream,
    ref_stream,
    car_stream,
    cdr_stream,
)


def integers_starting_from(n):
    return cons_stream(n, lambda: integers_starting_from(n + 1))


def divisible(x, y):
    return x % y == 0


integers = integers_starting_from(1)
no_sevens = filter_stream(lambda x: not divisible(x, 7), integers)

print(ref_stream(no_sevens, 100))

### 斐波那契数列的无穷流 

In [ ]:
from base_definition import cons_stream, ref_stream

def fibgen(a, b):
    return cons_stream(a, lambda : fibgen(b, a+b))

print(ref_stream(fibgen(0, 1), 3))

### 利用厄拉多塞筛法，构造素数的无穷流 

In [ ]:
from base_definition import (
    cons_stream,
    filter_stream,
    ref_stream,
    car_stream,
    cdr_stream,
)


def sieve(stream):
    return cons_stream(
        car_stream(stream),
        lambda: sieve(
            filter_stream(lambda x: x % car_stream(stream) != 0, cdr_stream(stream))
        ),
    )


print(ref_stream(sieve(integers_starting_from(2)), 50))

### 隐式地定义整数的一个无穷流


In [ ]:
from base_definition import cons_stream, map_stream, ref_stream


def ones():
    return cons_stream(1, lambda: ones())


def add_stream(s1, s2):
    return map_stream(lambda x1, x2: x1 + x2, s1, s2)


def integers():
    def integers_helper(current):
        value = current + 1
        return cons_stream(value, lambda: integers_helper(value))

    return integers_helper(0)


s = integers()
print(ref_stream(s, 2))
print(ref_stream(s, 10))

### 倍数增加stream  

In [ ]:
from base_definition import map_stream, delay, cons_stream, ref_stream


def scale_stream(stream, factor):
    return map_stream(lambda x: x * factor, stream)


def ones():
    return cons_stream(1, lambda: ones())


s = scale_stream(ones(), 2)

print(ref_stream(s, 3))

### 3.53 
2， 4， 8， 16， 32， 64

In [ ]:
from base_definition import add_stream, cons_stream, ref_stream


def double_sum():
    def double_sum_helper(current_sum):
        value = current_sum + current_sum
        return cons_stream(value, lambda: double_sum_helper(value))

    return double_sum_helper(1)


s = double_sum()
print(ref_stream(s, 1))
print(ref_stream(s, 3))
print(ref_stream(s, 6))

### 3.54 


In [ ]:
from base_definition import cons_stream, map_stream, cdr_stream


def integers_starting_from(n):
    return cons_stream(n, lambda: integers_starting_from(n + 1))


integers = integers_starting_from(1)


def factorials(s):

    def partial_mul_helper(s, current_mul):
        value = car_stream(s) * current_mul
        return cons_stream(
            value,
            lambda: partial_mul_helper(cdr_stream(s), value),
        )

    return partial_mul_helper(s, 1)


s = factorials(integers)

print(ref_stream(s, 1))
print(ref_stream(s, 5))

### 3.55 

In [ ]:
from base_definition import cons_stream, ref_stream, add_stream, cdr_stream, car_stream


def partial_sums(s):
    def partial_sums_helper(s, current_sum):
        value = car_stream(s) + current_sum
        return cons_stream(
            value,
            lambda: partial_sums_helper(cdr_stream(s), value),
        )

    return partial_sums_helper(s, 0)


def integers_starting_from(n):
    return cons_stream(n, lambda: integers_starting_from(n + 1))


integers = integers_starting_from(1)

t = partial_sums(integers)
print(ref_stream(t, 1))
print(ref_stream(t, 2))
print(ref_stream(t, 3))

In [ ]:
def partial_sums(stream):
    """Generate a stream of partial sums from the input stream."""
    total = 0
    for value in stream:
        total += value
        yield total

# Example usage:
def integers_starting_from(n):
    """Generate an infinite stream of integers starting from n."""
    while True:
        yield n
        n += 1

# Create an infinite stream of integers starting from 1
integers = integers_starting_from(1)

# Create a stream of partial sums
partial_sums_stream = partial_sums(integers)

# Print the first 10 elements of the partial sums stream
for _ in range(10):
    print(next(partial_sums_stream))


### 3.56 
wrong answers

In [17]:
from base_definition import map_stream, delay, cons_stream, ref_stream


def scale_stream(stream, factor):
    return map_stream(lambda x: x * factor, stream)


def merge_streams(s1, s2):
    if car_stream(s1) < car_stream(s2):
        return cons_stream(car_stream(s1), lambda: merge_streams(cdr_stream(s1), s2))
    elif car_stream(s1) > car_stream(s2):
        return cons_stream(car_stream(s2), lambda: merge_streams(s1, cdr_stream(s2)))
    else:
        return cons_stream(
            car_stream(s1), lambda: merge_streams(cdr_stream(s1), cdr_stream(s2))
        )


def hamming():
    def hamming_helper():
        return cons_stream(
            1,
            lambda: merge_streams(
                merge_streams(
                    scale_stream(hamming_stream, 2), scale_stream(hamming_stream, 3)
                ),
                scale_stream(hamming_stream, 5),
            ),
        )

    hamming_stream = hamming_helper()
    return hamming_stream

hamming_numbers = hamming()

print(ref_stream(hamming_numbers, 1))
print(ref_stream(hamming_numbers, 2))
print(ref_stream(hamming_numbers, 3))
print(ref_stream(hamming_numbers, 4))

1


RecursionError: maximum recursion depth exceeded

### 3.58  

In [19]:
def expand(num, den, radix):
    return cons_stream(
        (num * radix) // den, lambda: expand((num * radix) % den, den, radix)
    )


s1 = expand(1, 7, 10)
s2 = expand(3, 8, 10)

print(ref_stream(s1, 3))
print(ref_stream(s2, 3))

2
5


### 